### Format and combine frame txt files of yolo_traking 
path should be specified accordingly

In [1]:
import os
import re
from PIL import Image
from tqdm import tqdm

def get_image_dimensions(image_directory):
    example_image_path = os.path.join(image_directory, os.listdir(image_directory)[0])
    with Image.open(example_image_path) as img:
        width, height = img.size
    return width, height

def process_file(file_path, image_width, image_height):
    frame_number = os.path.basename(file_path).split('_')[-1].split('.')[0]
    output_data = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines
        if not lines:
            print(f"File {file_path} is empty.")
        for line in lines:
            data = line.strip().split()
            if len(data) == 6:
                track_id = data[-1]
                bbox_data = data[1:5]  # Ignore the first column
                x_center, y_center, width, height = map(float, bbox_data)
                
                # Convert normalized coordinates to pixel values
                x_center *= image_width
                y_center *= image_height
                width *= image_width
                height *= image_height
                
                # Convert center coordinates to top-left coordinates
                x_top_left = x_center - (width / 2)
                y_top_left = y_center - (height / 2)
                
                output_line = f"{frame_number},{track_id},{x_top_left:.2f},{y_top_left:.2f},{width:.2f},{height:.2f},1,-1,-1,-1"
                output_data.append(output_line)
    
    return output_data

def sorted_numerically(file_list):
    def numerical_sort(value):
        numbers = re.findall(r'\d+', value)
        return list(map(int, numbers)) if numbers else [0]
    return sorted(file_list, key=numerical_sort)

def process_video(video_name, input_directory, image_directory, output_directory):
    # Get the dimensions of one image
    image_width, image_height = get_image_dimensions(image_directory)
    
    output_file = os.path.join(output_directory, f"{video_name}.txt")
    
    label_directory = os.path.join(input_directory, 'labels')
    files = [file_name for file_name in os.listdir(label_directory) if file_name.startswith(video_name) and file_name.endswith('.txt')]
    files = sorted_numerically(files)
    
    all_data = []
    
    for file_name in files:
        file_path = os.path.join(label_directory, file_name)
        #print(f"Processing file: {file_path}")
        output_data = process_file(file_path, image_width, image_height)
        if not output_data:
            print(f"No data processed for file: {file_path}")
        all_data.extend(output_data)
    
    # Sort all data by frame number first, then by track ID
    all_data.sort(key=lambda x: (int(x.split(',')[0]), int(x.split(',')[1])))
    
    with open(output_file, 'w') as out_file:
        for line in all_data:
            out_file.write(line + '\n')
    
    print(f"Data has been written to {output_file}")

def main(dataset_dir, result_base_dir, output_base_dir):
    trackers = ['deepocsort', 'botsort', 'ocsort', 'bytetrack']
    
    for tracker in trackers:
        result_dir = os.path.join(result_base_dir, f"{tracker}__input_1280__conf_.25")
        output_dir = os.path.join(output_base_dir, f"{tracker.upper()}__input_1280__conf_.25")
        
        # Ensure the output directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        video_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
        
        for video_dir in tqdm(video_dirs, desc=f"Processing videos for {tracker}", unit="video"):
            video_name = video_dir
            input_directory = os.path.join(result_dir, video_name)
            image_directory = os.path.join(dataset_dir, video_name, 'img1')
            process_video(video_name, input_directory, image_directory, output_dir)

# Specify the dataset directory, base result directory, and base output directory
dataset_dir = '/workspace/LiteSORT/datasets/MOT20/train'
result_base_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/MOT20'
output_base_dir = '/workspace/LiteSORT/results/off-the-shelf/MOT20'

# Run the processing for all videos in the dataset for all trackers
main(dataset_dir, result_base_dir, output_base_dir)


Processing videos for deepocsort:   0%|          | 0/4 [00:00<?, ?video/s]

Processing videos for deepocsort:  25%|██▌       | 1/4 [00:00<00:01,  2.28video/s]

Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/DEEPOCSORT__input_1280__conf_.25/MOT20-02.txt


Processing videos for deepocsort:  50%|█████     | 2/4 [00:01<00:01,  1.70video/s]

Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/DEEPOCSORT__input_1280__conf_.25/MOT20-03.txt


...

Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/BYTETRACK__input_1280__conf_.25/MOT20-02.txt


Processing videos for bytetrack:  50%|█████     | 2/4 [00:00<00:00,  2.28video/s]

Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/BYTETRACK__input_1280__conf_.25/MOT20-03.txt


Processing videos for bytetrack: 100%|██████████| 4/4 [00:01<00:00,  3.24video/s]

Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/BYTETRACK__input_1280__conf_.25/MOT20-05.txt
Data has been written to /workspace/LiteSORT/results/off-the-shelf/MOT20/BYTETRACK__input_1280__conf_.25/MOT20-01.txt


### Visualize Formated txt
path should be specified accordingly

In [6]:
import os
import cv2

def draw_bounding_boxes(image_dir, result_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(result_file, 'r') as file:
        lines = file.readlines()
    
    bbox_data = {}
    for line in lines:
        data = line.strip().split(',')
        frame_id = int(data[0])
        track_id = int(data[1])
        x, y, w, h = map(float, data[2:6])
        
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        
        bbox_data[frame_id].append((track_id, x, y, w, h))
    
    for frame_id in bbox_data:
        image_path = os.path.join(image_dir, f"{frame_id:06d}.jpg")
        image = cv2.imread(image_path)
        
        for bbox in bbox_data[frame_id]:
            track_id, x, y, w, h = bbox
            top_left = (int(x), int(y))
            bottom_right = (int(x + w), int(y + h))
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
            cv2.putText(image, str(track_id), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        output_image_path = os.path.join(output_dir, f"{frame_id:06d}.jpg")
        cv2.imwrite(output_image_path, image)
        print(f"Processed frame {frame_id}")

# Example usage
image_dir = '/workspace/LiteSORT/datasets/MOT20/train/MOT20-05/img1'  # Adjust this to your image directory
result_file = '/workspace/LiteSORT/results/off-the-shelf/MOT20-train/DeepOC-SORT__input_1280__conf_.25/data/MOT20-05.txt'  # Adjust this to your result file
output_dir = '/workspace/LiteSORT/yolo_tracking/hbai_scripts/viz_MOT20/viz_deepocsort__input_1280__conf_.25/MOT20-05'  # Adjust this to your desired output directory

draw_bounding_boxes(image_dir, result_file, output_dir)


Processed frame 1
Processed frame 2
Processed frame 3
Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25
Processed frame 26
Processed frame 27
Processed frame 28
Processed frame 29
Processed frame 30
Processed frame 31
Processed frame 32
Processed frame 33
Processed frame 34
Processed frame 35
Processed frame 36
Processed frame 37
Processed frame 38
Processed frame 39
Processed frame 40
Processed frame 41
Processed frame 42
Processed frame 43
Processed frame 44
Processed frame 45
Processed frame 46
Processed frame 47
Processed frame 48
Processed frame 49
Processed frame 50
Processed frame 51
Processed frame 52
Processed frame 53
Pr